<a href="https://colab.research.google.com/github/boykeu/dataset/blob/master/codex_hc_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problems Statement

## Perusahaan X yang bergerak di produk digital, sedang berfokus pada pengelolaan talent nya. Manager HR merekrut Anda selaku Data Scientist untuk membantu dalam melihat kondisi talent perusahaan X berdasarkan data.

#### Fakta yang ada:

#### a. Untuk mengurangi cost dalam rekrutmen, Perusahaan X berupaya mempertahankan talent eksisting.

#### b. Sudah dilakukan pengukuran performansi, employee satisfaction dan employee engagement namun belum ada tindak lanjut dari hasil pengukuran. 


## Tugas Anda:

#### 1. Berikan insight ke Management terkait kondisi talent di perusahaan X dengan visualisasi profil talent (dalam bentuk ppt/ Tableau/ Power BI/ Google Data Studio atau platform visualisasi lainnya)

## Pilih minimal 1 dari pilihan berikut (buatlah dalam bentuk Power Point dengan rincian minimal terdapat rumusan masalah, metodologi, analisis, rekomendasi dan link git model):

#### 2. Bagaimana memprediksi talent yang berpotensi akan terminate (resign, performansi rendah dan lainnya)?

#### 3. Bagaimana membuat rekomendasi terkait kebutuhan development talent?.


#### Diperbolehkan menambahkan asumsi dan data lainnya di luar dataset untuk mempertajam analisis (jika dibutuhkan).


###### Dataset dapat diunduh melalui http://bit.ly/datasethrcodex

In [ ]:
# The libraries

import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
file_upload = files.upload()

In [ ]:
df = pd.read_csv('dataset HR analytics.csv')
df.head(10)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.isna().sum()

In [ ]:
#check missing /null values in the data

df.isnull().values.any()

In [ ]:
df.describe()

In [ ]:
# Create target field 'isAttrition'
df = df.assign(isAttrition=np.where(df['DateofTermination'].isnull(), False, True))

# Check how many employee terminated or still employed
df.isAttrition.value_counts()

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(32, 8.27)})

sns.countplot(df['isAttrition'])

In [ ]:
# Count no of employees that left and stayed by squad
sns.countplot(x = 'Squad', hue = 'isAttrition', data = df, palette = 'colorblind')

In [ ]:
# print all of the data types and their unique value counts

for column in df.columns:
  if df[column].dtype == object:
    print(str(column) + ' : ' + str(df[column].unique()))
    print(df[column].value_counts())
    print('___________________________________________________________________________')

In [ ]:
df.corr()

In [ ]:
sns.set(rc={'figure.figsize':(14,14)})
sns.heatmap(df.corr(), annot=True, fmt='.0%')

In [ ]:
# Transform the data (Non-numerical ---> Numerical)
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df = df.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
df